In [2]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge

## `HypothesisSpace` Class

An abstract class that defines a hypothesis space using two methods:

* `train(self, X, y)`: trains a the hypothesis on the given data `X` and labels `y` and returns the coeffcients of the trained model
* `assign(self, params)`: assigns a given parameters vector `params` to an instace of the model and returns a callabale

This class should be implemented for the desired models to work with.

In [3]:
class HypothesisSpace():
    
    def train(self, X, y):
        """
        X: np.ndarray of shape (number of samples, number of features)
        y: np.ndarray of shape (number of samples, )
        
        Returns: np.ndarray of shape (number of features, )
        """
        raise NotImplementedError("This method is not implemented")
    
    def assign(self, params):
        """
        params: np.ndarray of shape (number of features, )
        
        Returns: callable(x): h(x)
        """
        raise NotImplementedError("This method is not implemented")

## `BiasVariance` Class

A class that handles the calculation of the bias-variance estimates, initialized with only the target function and the desired hypothesis space.

public methods are:

* `bias2(self)`: returns an estimate of the squared bias
* `variance(self)`: returns an estimate of the variance

In [25]:
class BiasVariance():
    
    def __init__(self, target, hypothesis_class):
        """
        target: callabale(x): f(x)
        hypothesis_class: HypothesisSpace
        """
        
        self.target = target
        self.hypothesis_class = hypothesis_class()
        self.sample_x = np.arange(-1, 1, 0.01)
        self.h_bar = self._estimate_h_bar()
        
    def _estimate_h_bar(self):
        """
        estiamtes the parameters vector of the mean hypothesis
        and returns an instance of the hypothesis space with that estimate
        """
        
        datasets_mean_coef = None  # holds the average of the parameters vector across the datasets
        
        # seed the random generators with a known value to get the same
        # simulated datasets when estimating the variance
        np.random.seed(1)
        
        for i in range(1000):
            
            # generate a simulated dataset
            X = np.random.uniform(-1, 1, 200)
            noise = np.random.normal(scale=2, size=200)
            y = self.target(X) + noise
            
            trained_coef = self.hypothesis_class.train(X, y)
            if datasets_mean_coef is None:
                datasets_mean_coef = trained_coef
            else:
                datasets_mean_coef += trained_coef
                
        datasets_mean_coef /= 1000.0  # finalize the average estimation
        
        return self.hypothesis_class.assign(datasets_mean_coef)
    
    def bias2(self):
        """
        returns an estimate of the squared bias
        """
        
        bias2 = np.mean((self.h_bar(self.sample_x) - self.target(self.sample_x)) ** 2)
        
        return bias2
    
    def variance(self):
        """
        returns an estimate of the variance
        """
        
        datasets_variances = []  # holds the variance estimate for each simulated dataset
        
        np.random.seed(1)
        
        for i in range(1000):
            
            # generate a simulated dataset
            X = np.random.uniform(-1, 1, 200)
            noise = np.random.normal(scale=2, size=200)
            y = self.target(X) + noise
            
            trained_coef = self.hypothesis_class.train(X, y)
            trained_instance = self.hypothesis_class.assign(trained_coef)
            
            var = np.mean((trained_instance(X) - self.h_bar(X)) ** 2)
            datasets_variances.append(var)
            
        return np.mean(datasets_variances)

## Unregularized 10th Degree Model

In [26]:
class UnRegularized10thModel(HypothesisSpace):
    
    def train(self, X, y):
        transform = PolynomialFeatures(10)
        trainer = LinearRegression()
        X = np.reshape(X, (-1, 1))
        
        poly_X = transform.fit_transform(X)
        trainer.fit(poly_X, y)
        params = trainer.coef_
        params[0] = trainer.intercept_
        
        return params
    
    def assign(self, params):
        intercept = params[0]
        coef = params
        coef[0] = 0.
        
        transform = PolynomialFeatures(10)
        model = LinearRegression()
        model.coef_ = coef
        model.intercept_ = intercept
        
        return lambda x: model.predict(transform.fit_transform(np.reshape(x, (-1, 1))))

In [27]:
unregularizedBV = BiasVariance(lambda x: np.sin(np.pi * x), UnRegularized10thModel)

bias2 = unregularizedBV.bias2()
variance = unregularizedBV.variance()

print "Estimated Squred Bias: %.6f" % (bias2)
print "Estimated Variance: %.6f" % (variance)
print "Estimated Risk: %.6f" % (bias2 + variance + 4) 

Estimated Squred Bias: 0.000268
Estimated Variance: 0.219390
Estimated Risk: 4.219657


## Regularized 10th Degree Model

In [32]:
class Regularized10thModel(HypothesisSpace):
    
    def train(self, X, y):
        transform = PolynomialFeatures(10)
        trainer = Ridge(alpha=2)
        X = np.reshape(X, (-1, 1))
        
        poly_X = transform.fit_transform(X)
        trainer.fit(poly_X, y)
        params = trainer.coef_
        params[0] = trainer.intercept_
        
        return params
    
    def assign(self, params):
        intercept = params[0]
        coef = params
        coef[0] = 0.
        
        transform = PolynomialFeatures(10)
        model = Ridge(alpha=2)
        model.coef_ = coef
        model.intercept_ = intercept
        
        return lambda x: model.predict(transform.fit_transform(np.reshape(x, (-1, 1))))

In [33]:
regularizedBV = BiasVariance(lambda x: np.sin(np.pi * x), Regularized10thModel)

bias2 = regularizedBV.bias2()
variance = regularizedBV.variance()

print "Estimated Squred Bias: %.6f" % (bias2)
print "Estimated Variance: %.6f" % (variance)
print "Estimated Risk: %.6f" % (bias2 + variance + 4) 

Estimated Squred Bias: 0.034948
Estimated Variance: 0.083310
Estimated Risk: 4.118258
